In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  # Use scikeras for scikit-learn compatibility
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings
import time
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV


In [2]:
#importing the weather dataframe
path = r'C:\Users\karim\Documents\machine learning'
X= pd.read_csv(os.path.join(path, 'data', 'prepared Data', 'machine2_weather_clean.csv'))

In [143]:
df_answers = pd.read_csv(os.path.join(path, 'data', 'original Data', 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))

In [145]:
X

,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,...,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max,KASSEL_cloud_cover,MUNCHENB_pressure,STOCKHOLM_humidity
0,7,0.85,1.0180,0.32,0.09,0.7,6.5,0.8,10.9,1,...,1.0003,0.45,0.34,4.7,8.5,6.0,10.9,8,1.0304,0.98
1,6,0.84,1.0180,0.36,1.05,1.1,6.1,3.3,10.1,6,...,1.0007,0.25,0.84,0.7,8.9,5.6,12.1,6,1.0292,0.62
2,8,0.90,1.0180,0.18,0.30,0.0,8.5,5.1,9.9,6,...,1.0096,0.17,0.08,0.1,10.5,8.1,12.9,8,1.0320,0.69
3,3,0.92,1.0180,0.58,0.00,4.1,6.3,3.8,10.6,8,...,1.0184,0.13,0.98,0.0,7.4,7.3,10.6,6,1.0443,0.98
4,6,0.95,1.0180,0.65,0.14,5.4,3.0,-0.7,6.0,8,...,1.0328,0.46,0.00,5.7,5.7,3.0,8.4,7,1.0430,0.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22945,1,0.79,1.0248,1.34,0.22,7.7,15.9,11.4,21.4,2,...,1.0142,1.13,0.41,3.4,10.7,7.9,13.5,4,1.0263,0.98
22946,6,0.77,1.0244,1.34,0.22,5.4,16.7,14.3,21.9,0,...,1.0142,1.13,0.41,3.4,10.7,7.9,13.5,3,1.0263,1.00
22947,4,0.76,1.0227,1.34,0.22,6.1,16.7,13.1,22.4,2,...,1.0142,1.13,0.41,3.4,10.7,7.9,13.5,3,1.0263,0.85
22948,5,0.80,1.0212,1.34,0.22,5.8,15.4,11.6,21.1,1,...,1.0142,1.13,0.41,3.4,10.7,7.9,13.5,3,1.0263,0.94


In [147]:
df_answers.head(100)

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,19600405,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
96,19600406,1,0,0,1,1,0,1,0,1,1,0,0,0,0,0
97,19600407,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0
98,19600408,1,1,0,0,0,0,0,1,0,1,1,0,0,0,0


In [149]:
#dropping date from df_answers
y=df_answers.drop(columns=['DATE'],axis=1)

# Reshaping and Splitting

In [152]:

y.head(100)

,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0
96,1,0,0,1,1,0,1,0,1,1,0,0,0,0,0
97,1,0,0,0,0,0,0,1,0,1,1,0,0,0,0
98,1,1,0,0,0,0,0,1,0,1,1,0,0,0,0


In [154]:

X_ = np.array(X)
y = np.array(y)

In [156]:
X_ = X_.reshape(-1,15,9)
X_.shape

(22950, 15, 9)

In [158]:
# Use argmax to transform y

y =  np.argmax(y, axis = 1)


In [160]:
 #Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X_,y,random_state = 42)

In [162]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


In [164]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 #_count_classes(Y_train)
# Make scorer accuracy

score_acc = make_scorer(accuracy_score)

In [166]:
print(y_train.shape, y_test.shape)
print(np.unique(y_train))
print(np.isnan(y_train).any())

(17212,) (5738,)
[ 0  1  2  3  4  5  6  7  8  9 10 11 13 14]
False


# Bayesian Hyperparameter Optimization

In [169]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, params={'callbacks':[es]}).mean()
    return score

In [171]:
start = time.time()
params ={
    'neurons': (10, 64),
    'kernel': (1, 3),
    'activation':(0, 9), #9
    'optimizer':(0,7), #7
   'learning_rate':(0.01, 1),
    'batch_size': (100, 356), #(10, 50), #
    'epochs':(10, 30),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=3, n_iter=2) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   |  neurons  |  kernel   | activa... | optimizer | learni... | batch_... |  epochs   |  layers1  |  layers2  | normal... |  dropout  | dropou... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


C:\Users\karim\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\karim\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/11
99/99 - 4s - 45ms/step - accuracy: 0.6440 - loss: nan
Epoch 2/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 3/11
99/99 - 1s - 14ms/step - accuracy: 0.6440 - loss: nan
Epoch 4/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/11
99/99 - 1s - 14ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
25/25 - 1s - 28ms/step


C:\Users\karim\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\karim\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/11
99/99 - 4s - 43ms/step - accuracy: 0.6440 - loss: nan
Epoch 2/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 3/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 4/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 5/11
99/99 - 1s - 14ms/step - accuracy: 0.6440 - loss: nan
Epoch 6/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 7/11
99/99 - 1s - 14ms/step - accuracy: 0.6440 - loss: nan
Epoch 8/11
99/99 - 3s - 26ms/step - accuracy: 0.6440 - loss: nan
Epoch 9/11
99/99 - 1s - 14ms/step - accuracy: 0.6440 - loss: nan
Epoch 10/11
99/99 - 1s - 13ms/step - accuracy: 0.6440 - loss: nan
Epoch 11/11
99/99 - 1s - 12ms/step - accuracy: 0.6440 - loss: nan
25/25 - 2s - 72ms/step


...

Epoch 1/10
47/47 - 8s - 179ms/step - accuracy: 0.6324 - loss: 2.6987
Epoch 2/10
47/47 - 1s - 29ms/step - accuracy: 0.6439 - loss: 2.6911
Epoch 3/10
47/47 - 1s - 27ms/step - accuracy: 0.6439 - loss: 2.6844
Epoch 4/10
47/47 - 1s - 29ms/step - accuracy: 0.6439 - loss: 2.6788
Epoch 5/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6738
Epoch 6/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6691
Epoch 7/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6646
Epoch 8/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6601
Epoch 9/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6556
Epoch 10/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6510
12/12 - 1s - 61ms/step


C:\Users\karim\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\karim\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
47/47 - 12s - 245ms/step - accuracy: 0.6306 - loss: 2.7002
Epoch 2/10
47/47 - 3s - 56ms/step - accuracy: 0.6440 - loss: 2.6910
Epoch 3/10
47/47 - 1s - 28ms/step - accuracy: 0.6440 - loss: 2.6843
Epoch 4/10
47/47 - 1s - 28ms/step - accuracy: 0.6440 - loss: 2.6786
Epoch 5/10
47/47 - 1s - 30ms/step - accuracy: 0.6440 - loss: 2.6735
Epoch 6/10
47/47 - 1s - 28ms/step - accuracy: 0.6440 - loss: 2.6686
Epoch 7/10
47/47 - 1s - 28ms/step - accuracy: 0.6440 - loss: 2.6639
Epoch 8/10
47/47 - 1s - 29ms/step - accuracy: 0.6440 - loss: 2.6590
Epoch 9/10
47/47 - 1s - 28ms/step - accuracy: 0.6440 - loss: 2.6540
Epoch 10/10
47/47 - 1s - 29ms/step - accuracy: 0.6440 - loss: 2.6487
12/12 - 1s - 56ms/step


C:\Users\karim\anaconda3\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\karim\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
47/47 - 9s - 182ms/step - accuracy: 0.6316 - loss: 2.7008
Epoch 2/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6914
Epoch 3/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6849
Epoch 4/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6795
Epoch 5/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6747
Epoch 6/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6704
Epoch 7/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6664
Epoch 8/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6625
Epoch 9/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6586
Epoch 10/10
47/47 - 1s - 28ms/step - accuracy: 0.6439 - loss: 2.6547
12/12 - 1s - 56ms/step
| 5         | 0.6439693 | 64.0      | 3.0       | 9.0       | 7.0       | 1.0       | 294.28363 | 10.0      | 3.0       | 3.0       | 1.0       | 1.0       | 0.3       |
Search took 12.10523529847463 minutes


In [173]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
optimum['activation'] = activationL[round(optimum['activation'])]
optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])
optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']
optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
             'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
             'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
             'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'neurons': 55,
 'kernel': 1.5659289131547387,
 'activation': 'softplus',
 'optimizer': <keras.src.optimizers.sgd.SGD at 0x1b7993b2de0>,
 'learning_rate': 0.32495292690944505,
 'batch_size': 234,
 'epochs': 18,
 'layers1': 2,
 'layers2': 2,
 'normalization': 0.26844797049470503,
 'dropout': 0.432506546406474,
 'dropout_rate': 0.12293123711499533}

In [175]:
# Define shapes
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])

# Hyperparameters
n_hidden = 32
neurons = 55
kernel = 2  # round(1.7745714671649524)
activation = 'softplus'
learning_rate = 0.3
batch_size = 234
epochs = 18
layers1 = 2
layers2 = 2
normalization = 0.2
dropout = 0.4
dropout_rate = 0.12

optimizer = Adagrad(learning_rate=learning_rate)

# CNN model
model = Sequential()

model.add(Conv1D(filters=64, kernel_size=int(kernel), activation=activation, input_shape=(timesteps, input_dim)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=128, kernel_size=3, activation=activation))
model.add(MaxPooling1D(pool_size=2))

if normalization > 0.5:
    model.add(BatchNormalization())

model.add(Flatten())

# Add dense layers
for _ in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for _ in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(Dense(15, activation='softmax'))  # assuming 15 classes

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


C:\Users\karim\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_51"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_52 (Conv1D)              │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_52 (MaxPooling1D) │ (None, 7, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_53 (Conv1D)              │ (None, 5, 128)         │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_53 (MaxPooling1D) │ (None, 2, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_51 (Flatten)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_256 (Dense)               │ (None, 55)             │        14,135 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_257 (Dense)               │ (None, 55)             │         3,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_258 (Dense)               │ (None, 55)             │         3,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_259 (Dense)               │ (None, 55)             │         3,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_260 (Dense)               │ (None, 15)             │           840 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,135 (195.84 KB)

 Trainable params: 50,135 (195.84 KB)

 Non-trainable params: 0 (0.00 B)

In [177]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)
acc = model.evaluate(X_train,y_train)
print('Loss:', acc[0], 'Accuracy', acc[1])

Epoch 1/18
74/74 - 3s - 45ms/step - accuracy: 0.5808 - loss: 43.4371
Epoch 2/18
74/74 - 1s - 18ms/step - accuracy: 0.6440 - loss: 1.1650
Epoch 3/18
74/74 - 1s - 19ms/step - accuracy: 0.6440 - loss: 1.1390
Epoch 4/18
74/74 - 1s - 17ms/step - accuracy: 0.6440 - loss: 1.0538
Epoch 5/18
74/74 - 1s - 14ms/step - accuracy: 0.6409 - loss: 1.0225
Epoch 6/18
74/74 - 1s - 20ms/step - accuracy: 0.6409 - loss: 1.0049
Epoch 7/18
74/74 - 1s - 15ms/step - accuracy: 0.6417 - loss: 0.9758
Epoch 8/18
74/74 - 1s - 17ms/step - accuracy: 0.6427 - loss: 0.9665
Epoch 9/18
74/74 - 1s - 15ms/step - accuracy: 0.6453 - loss: 0.9556
Epoch 10/18
74/74 - 1s - 17ms/step - accuracy: 0.6506 - loss: 0.9427
Epoch 11/18
74/74 - 1s - 17ms/step - accuracy: 0.6559 - loss: 0.9310
Epoch 12/18
74/74 - 1s - 18ms/step - accuracy: 0.6577 - loss: 0.9258
Epoch 13/18
74/74 - 1s - 18ms/step - accuracy: 0.6392 - loss: 1.0177
Epoch 14/18
74/74 - 1s - 17ms/step - accuracy: 0.6602 - loss: 0.9258
Epoch 15/18
74/74 - 1s - 17ms/step - accur

# confusion matrix

In [200]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)
    
    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)
        
    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])
    
    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [202]:
#3rd confusion matrix result
print(confusion_matrix(y_test, model.predict(X_test),stations))

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Pred        BASEL  BELGRADE  HEATHROW  MADRID
True                                         
BASEL        3248       371         0      63
BELGRADE      436       602         0      54
BUDAPEST       91        99         0      24
DEBILT         40        37         0       5
DUSSELDORF     17        10         0       2
HEATHROW       32        27         1      22
KASSEL          4         7         0       0
LJUBLJANA      19        33         0       9
MAASTRICHT      3         2         0       4
MADRID         83        60         1     314
MUNCHENB        3         5         0       0
OSLO            2         2         0       1
STOCKHOLM       0         3         0       1
VALENTIA        0         0         0       1
